In [1]:
%matplotlib qt


In [2]:
#import stuffs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import ndimage, signal
import os, fnmatch, glob, re, time
from scipy.io import loadmat, savemat
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import colors
from matplotlib.widgets import RectangleSelector
import cv2
from PIL import Image


# Image Slices Viewer


Scroll through 2D image slices of a 3D array. https://matplotlib.org/2.1.2/gallery/animation/image_slices_viewer.html



In [ ]:
from __future__ import print_function

class IndexTracker(object):
    def __init__(self, ax, X, wv, info = pd.DataFrame({"exposure" : [" "], "power" : [" "], "chipname" : [" "]}),\
                 wv_min = 0, wv_max = 1, wv_step = 1, cmin = 0, cmax = 10000):
        self.ax = ax
        self.wv_min = wv_min
        self.wv_max = wv_max
        self.wv_step = wv_step
        self.wv = wv
        
        ax.set_title('{chipname} at {power}% laser power, {exposure} ms exposure time'.format(chipname = info["chipname"].item(),\
                    power = info["power"].item(), exposure = info["exposure"].item()))
        
       
        
        self.X = X
        self.slices, rows, cols = X.shape
        #start with first slice
        self.ind = 0

        self.im = ax.imshow(self.X[self.ind, :, : ], cmap = 'gray')
        plt.colorbar(self.im, label = "[a.u.]")#, ax = self.ax)
        self.im.set_clim(vmin = cmin, vmax = cmax)    
        self.update()
        
        
        
        self.rect = RectangleSelector(self.ax, self.line_select_callback,
                                       drawtype='box', useblit=True,
                                       button=[1, 3],  # don't use middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True)

        plt.connect('key_press_event', self.rect)
        
        #Create place holder for spectrum and second figure
        self.spectrum = np.zeros(self.X.shape[0])
        self.fig2, self.ax2 = plt.subplots(1,1)
        self.ax2.set_title('{chipname} at {power}% laser power, {exposure} ms exposure time'.format(chipname = info["chipname"].item(),\
                    power = info["power"].item(), exposure = info["exposure"].item()))
        self.ax2.plot(range(self.wv_min, self.wv_max +1), self.spectrum)
        self.ax2.set_xlabel("Wavelength[nm]")
        self.ax2.set_ylabel("Signal [a.u.]")
        

    def onscroll(self, event):
        #print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[self.ind, :, :])
        ax.set_ylabel('Wavelength {wv} nm'.format(wv = self.wv[self.ind]))
        self.im.axes.figure.canvas.draw()
        
    def line_select_callback(self, eclick, erelease):
        #'eclick and erelease are the press and release events'
        #x and y are interchanged, because the first dimension is the rows, therefore in representation it becomes the y
        self.y1, self.x1 = eclick.xdata, eclick.ydata
        self.y2, self.x2 = erelease.xdata, erelease.ydata
        print("Click Coordinate (%3.2f, %3.2f) --> Release Coordinate(%3.2f, %3.2f)" % (eclick.xdata, eclick.ydata, erelease.xdata, erelease.ydata))
        print("The average intensity is %3.2f" % (np.nanmean(self.X[self.ind, \
                                                                int(self.x1):int(self.x2),
                                                                int(self.y1):int(self.y2)])))
        self.spectrum = np.array([np.nanmean(self.X[i, int(self.x1):int(self.x2), int(self.y1):int(self.y2)]) for i in range(self.X.shape[0])])

        self.ax2.plot(range(self.wv_min, self.wv_max +1), self.spectrum, label = "[%.0f:%.0f, %.0f:%.0f]" % (self.x1, self.x2, self.y1, self.y2))
        self.ax2.legend()
        self.fig2.canvas.draw()
        #print(" The button you used were: %s %s" % (eclick.button, erelease.button))

        
        

# Load Data

In [ ]:


#Load Before Datacube
folder_name = "1"
start_wavelength = 651
end_wavelength = 660

spectra = loadmat("./CS766ProjectDataset/{folder}/Before/{start_wv}.mat".format(start_wv = start_wavelength, folder = folder_name))
spectra = np.array(spectra['spectra'])
data_cube = [spectra]
for i in range(start_wavelength + 1,end_wavelength +1):
    #placeholder
    a = loadmat("./CS766ProjectDataset/{folder}/Before/{start_wv}.mat".format(start_wv = i, folder = folder_name))
    data_cube.append(np.array(a['spectra']))
#convert to np array
before = np.array(data_cube)

#load After Datacube

spectra = loadmat("./CS766ProjectDataset/{folder}/After/{start_wv}.mat".format(start_wv = start_wavelength, folder = folder_name))
spectra = np.array(spectra['spectra'])
data_cube = [spectra]
for i in range(start_wavelength + 1,end_wavelength +1):
    #placeholder
    a = loadmat("./CS766ProjectDataset/{folder}/After/{start_wv}.mat".format(start_wv = i, folder = folder_name))
    data_cube.append(np.array(a['spectra']))
#convert to np array
after = np.array(data_cube)

# Visualize Data

In [ ]:
data = before
data_shape = before.shape
wv = [i for i in range(start_wavelength,end_wavelength+1)]

fig, ax = plt.subplots(1, 1)


tracker = IndexTracker(ax, np.resize(data[np.isfinite(data)], data_shape), wv,  \
wv_min = start_wavelength, wv_max = end_wavelength, wv_step = 1,\
cmin = 0, cmax = 10)#3 * np.mean(data[np.isfinite(data)]))


fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()